In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.metrics import log_loss, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["figure.figsize"] = (20,3)

In [2]:
class evaluation:
    def __init__(self, true_s):
        self.true_df = pd.DataFrame(true_s)
        self.true_df.columns = ['swell']
        self.true_df['swell(t+1)'] = self.true_df['swell'].shift(-1)
        self.true_df['swell(t+2)'] = self.true_df['swell'].shift(-2)
        self.true_df['swell(t-1)'] = self.true_df['swell'].shift(1)
        self.true_df['swell(t-2)'] = self.true_df['swell'].shift(2)
    
        self.swell_columns = ['swell(t+1)','swell(t+2)','swell(t-1)','swell(t-2)']
    
    def _eval_one(self, pred):
        true = self.true_df.loc[pred.name]
        swell = true['swell'] 
        p_swell = pred['pred']

        if swell == 0 and p_swell == 0:
            return 1

        if swell == 1 and p_swell == 1:
            return 2

        if swell == 0 and p_swell == 1:
            if (true[self.swell_columns].values == 1).any():
                return 0
            else:
                return -1

        if swell == 1 and p_swell == 0:
            if (pred[self.swell_columns].values == 1).any():
                return 0
            else: 
                return -2
    
    def evaluate(self, pred):
        pred_df = pd.DataFrame(pred)
        pred_df.columns = ['pred']
        pred_df['swell(t+1)'] = pred_df['pred'].shift(-1)
        pred_df['swell(t+2)'] = pred_df['pred'].shift(-2)
        pred_df['swell(t-1)'] = pred_df['pred'].shift(1)
        pred_df['swell(t-2)'] = pred_df['pred'].shift(2)

        result = []
        for index in pred_df.index:
            result.append(self._eval_one(pred_df.loc[index]))
        
        #print(result)
        print(pd.Series(result).value_counts())
        
        return np.sum(result)

In [3]:
num_values = 7
degrees_week = np.linspace(0, 360, num_values + 1)[:-1]
print(degrees_week)
sin_week = np.sin(np.deg2rad(degrees_week))
print(sin_week)
cos_week = np.cos(np.deg2rad(degrees_week))
print(cos_week)

num_values = 12
degrees_month = np.linspace(0, 360, num_values + 1)[:-1]
print(degrees_month)
sin_month = np.sin(np.deg2rad(degrees_month))
print(sin_month)
cos_month = np.cos(np.deg2rad(degrees_month))
print(cos_month)

num_values = 31
degrees_day = np.linspace(0, 360, num_values + 1)[:-1]
print(degrees_day)
sin_day = np.sin(np.deg2rad(degrees_day))
print(sin_day)
cos_day = np.cos(np.deg2rad(degrees_day))
print(cos_day)

num_values = 24
degrees_hour = np.linspace(0, 360, num_values + 1)[:-1]
print(degrees_hour)
sin_hour = np.sin(np.deg2rad(degrees_hour))
print(sin_hour)
cos_hour = np.cos(np.deg2rad(degrees_hour))
print(cos_hour)

[   0.           51.42857143  102.85714286  154.28571429  205.71428571
  257.14285714  308.57142857]
[ 0.          0.78183148  0.97492791  0.43388374 -0.43388374 -0.97492791
 -0.78183148]
[ 1.          0.6234898  -0.22252093 -0.90096887 -0.90096887 -0.22252093
  0.6234898 ]
[   0.   30.   60.   90.  120.  150.  180.  210.  240.  270.  300.  330.]
[  0.00000000e+00   5.00000000e-01   8.66025404e-01   1.00000000e+00
   8.66025404e-01   5.00000000e-01   1.22464680e-16  -5.00000000e-01
  -8.66025404e-01  -1.00000000e+00  -8.66025404e-01  -5.00000000e-01]
[  1.00000000e+00   8.66025404e-01   5.00000000e-01   6.12323400e-17
  -5.00000000e-01  -8.66025404e-01  -1.00000000e+00  -8.66025404e-01
  -5.00000000e-01  -1.83697020e-16   5.00000000e-01   8.66025404e-01]
[   0.           11.61290323   23.22580645   34.83870968   46.4516129
   58.06451613   69.67741935   81.29032258   92.90322581  104.51612903
  116.12903226  127.74193548  139.35483871  150.96774194  162.58064516
  174.19354839  185.806

In [4]:
def make_coord(date,num_class):
    # 1 ~ 12, 1 ~ 7, 1 ~ 31
    degrees = np.linspace(0,360,num_values + 1)[:-1]
    sin = np.sin(np.deg2rad(degrees))
    cos = np.cos(np.deg2rad(degrees))
    return sin, cos


def date2coord(date, sin, cos):
    return cos[date-1], sin[date-1]


def date2week(x):
    y = int(str(x)[:4])
    m = int(str(x)[4:6])
    d = int(str(x)[6:8])
    day = datetime(y, m, d)
    return day.weekday()

In [5]:
df = pd.read_csv('./merged_info_data.tsv',sep='\t',index_col='Unnamed: 0')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34992 entries, 0 to 34991
Data columns (total 65 columns):
hour                     34992 non-null int64
climate                  34992 non-null object
weather_deterioration    34992 non-null int64
swell                    34392 non-null float64
21229_GUST풍속 1(m/s)      8637 non-null float64
21229_GUST풍속(m/s)        26091 non-null float64
21229_기온(°C)             33915 non-null float64
21229_수온(°C)             34629 non-null float64
21229_습도(%)              33939 non-null float64
21229_유의파고(m)            34700 non-null float64
21229_일시                 34817 non-null float64
21229_최대파고(m)            34788 non-null float64
21229_파주기(sec)           34800 non-null float64
21229_파향(deg)            34808 non-null float64
21229_평균파고(m)            34606 non-null float64
21229_풍속(m/s)            26122 non-null float64
21229_풍속1(m/s)           8638 non-null float64
21229_풍향(deg)            26121 non-null float64
21229_풍향1(deg)           8639 non-

In [6]:
df['swell'].value_counts()

0.0    31800
1.0     2592
Name: swell, dtype: int64

In [7]:
df['swell'] = df['swell'] + df['weather_deterioration']

In [8]:
df['swell'].value_counts()

0.0    26863
1.0     4937
2.0     2592
Name: swell, dtype: int64

In [9]:
df['month'] = df['hour'].apply(lambda x:int(str(x)[4:6]))
df['day'] = df['hour'].apply(lambda x:int(str(x)[6:8]))
df['day_hour'] = df['hour'].apply(lambda x:int((str(x)[8:])))
df['week'] = df['hour'].apply(date2week)
df['week'] = df['week'] + 1 
df['year'] = df['hour'].apply(lambda x:int(str(x)[:4]))

In [10]:
df['day_coord'] = df['day'].apply(date2coord, args=(sin_day, cos_day,))
df['month_coord'] = df['month'].apply(date2coord, args=(sin_month, cos_month))
df['week_coord'] = df['week'].apply(date2coord, args=(sin_week, cos_week,))
df['hour_coord'] = df['day_hour'].apply(date2coord, args=(sin_hour, cos_hour,))

In [11]:
s = (df[df['swell'].isnull()]['hour']//100).value_counts()
s

20170315    17
20171211    17
20161130    17
20170331    17
20141023    17
20161020    17
20140518    17
20170212    17
20150718    17
20140706    17
20150627    17
20160830    17
20171023    17
20171012    17
20141221    17
20170916    17
20160122    17
20150404    17
20150113    17
20140925    17
20170603    17
20161226    17
20151213    17
20160304    17
20151113    17
20160831     7
20170604     7
20171024     7
20140926     7
20171013     7
20150114     7
20141222     7
20151214     7
20150405     7
20150628     7
20140707     7
20150719     7
20170401     7
20171212     7
20170917     7
20151114     7
20170213     7
20140519     7
20160305     7
20161021     7
20170316     7
20161227     7
20161201     7
20160123     7
20141024     7
Name: hour, dtype: int64

In [12]:
s[s > 10].index.tolist()[0]

20170315

In [13]:
def get_prev_date(today):
    today = date(int(today[:4]),int(today[4:6]),int(today[6:]))
    return (today - timedelta(1)).strftime('%Y%m%d%H')

In [14]:
get_prev_date('20170101')

'2016123100'

In [15]:
test_date_hour = []
for d in s[s>10].index.astype(str):
    day = get_prev_date(d)
    for i in range(24):
        test_date_hour.append(str(int(day) + i))
len(test_date_hour)

600

In [16]:
df['swell(t+1)'] = df['swell'].shift(-1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34992 entries, 0 to 34991
Data columns (total 75 columns):
hour                     34992 non-null int64
climate                  34992 non-null object
weather_deterioration    34992 non-null int64
swell                    34392 non-null float64
21229_GUST풍속 1(m/s)      8637 non-null float64
21229_GUST풍속(m/s)        26091 non-null float64
21229_기온(°C)             33915 non-null float64
21229_수온(°C)             34629 non-null float64
21229_습도(%)              33939 non-null float64
21229_유의파고(m)            34700 non-null float64
21229_일시                 34817 non-null float64
21229_최대파고(m)            34788 non-null float64
21229_파주기(sec)           34800 non-null float64
21229_파향(deg)            34808 non-null float64
21229_평균파고(m)            34606 non-null float64
21229_풍속(m/s)            26122 non-null float64
21229_풍속1(m/s)           8638 non-null float64
21229_풍향(deg)            26121 non-null float64
21229_풍향1(deg)           8639 non-

In [17]:
df.columns

Index(['hour', 'climate', 'weather_deterioration', 'swell',
       '21229_GUST풍속 1(m/s)', '21229_GUST풍속(m/s)', '21229_기온(°C)',
       '21229_수온(°C)', '21229_습도(%)', '21229_유의파고(m)', '21229_일시',
       '21229_최대파고(m)', '21229_파주기(sec)', '21229_파향(deg)', '21229_평균파고(m)',
       '21229_풍속(m/s)', '21229_풍속1(m/s)', '21229_풍향(deg)', '21229_풍향1(deg)',
       '21229_현지기압(hPa)', '22105_GUST풍속 1(m/s)', '22105_GUST풍속(m/s)',
       '22105_기온(°C)', '22105_수온(°C)', '22105_습도(%)', '22105_유의파고(m)',
       '22105_일시', '22105_최대파고(m)', '22105_파주기(sec)', '22105_파향(deg)',
       '22105_평균파고(m)', '22105_풍속(m/s)', '22105_풍속1(m/s)', '22105_풍향(deg)',
       '22105_풍향1(deg)', '22105_현지기압(hPa)', '22106_GUST풍속 1(m/s)',
       '22106_GUST풍속(m/s)', '22106_기온(°C)', '22106_수온(°C)', '22106_습도(%)',
       '22106_유의파고(m)', '22106_일시', '22106_최대파고(m)', '22106_파주기(sec)',
       '22106_파향(deg)', '22106_평균파고(m)', '22106_풍속(m/s)', '22106_풍속1(m/s)',
       '22106_풍향(deg)', '22106_풍향1(deg)', '22106_현지기압(hPa)', '22190_일시',
   

In [18]:
def output_change(x):
    if x == 2:
        return 1
    else:
        return 0

In [19]:
df['hour'] = df['hour'].astype(str)
df.set_index('hour',inplace=True)

In [20]:
scale_features = ['swell','21229_파주기(sec)','22105_파주기(sec)','22106_파주기(sec)']
#['swell']#,'22453_파주기(sec)','22106_파주기(sec)','22106_현지기압(hPa)']
nor_features = ['hour_coord','day_coord','month_coord']#,'hour_coord']
target = ['swell(t+1)']

In [21]:
minmax_scaler = MinMaxScaler(copy=True, feature_range=(0,1))
temp_df = df[scale_features+nor_features+target]

minmax_scaler.fit(temp_df[scale_features].dropna())
    
train = temp_df[scale_features+nor_features + target].drop(test_date_hour).dropna()
test = temp_df[scale_features+nor_features + target].loc[test_date_hour]
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32199 entries, 2014010407 to 2017123123
Data columns (total 8 columns):
swell             32199 non-null float64
21229_파주기(sec)    32199 non-null float64
22105_파주기(sec)    32199 non-null float64
22106_파주기(sec)    32199 non-null float64
hour_coord        32199 non-null object
day_coord         32199 non-null object
month_coord       32199 non-null object
swell(t+1)        32199 non-null float64
dtypes: float64(5), object(3)
memory usage: 2.2+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 2017031400 to 2015111223
Data columns (total 8 columns):
swell             600 non-null float64
21229_파주기(sec)    599 non-null float64
22105_파주기(sec)    576 non-null float64
22106_파주기(sec)    587 non-null float64
hour_coord        600 non-null object
day_coord         600 non-null object
month_coord       600 non-null object
swell(t+1)        600 non-null float64
dtypes: float64(5), object(3)
memory usage: 42.2+ KB


In [22]:
test.fillna(method='ffill',inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 2017031400 to 2015111223
Data columns (total 8 columns):
swell             600 non-null float64
21229_파주기(sec)    600 non-null float64
22105_파주기(sec)    600 non-null float64
22106_파주기(sec)    600 non-null float64
hour_coord        600 non-null object
day_coord         600 non-null object
month_coord       600 non-null object
swell(t+1)        600 non-null float64
dtypes: float64(5), object(3)
memory usage: 42.2+ KB


In [23]:
train_X = minmax_scaler.transform(train[scale_features].values)

x_list = []
for col in nor_features:
    X_arr = np.array(train[col].apply(lambda x:np.array(x)).tolist())
    x_list.append(X_arr)

if len(nor_features) > 0:
    nor_X = np.concatenate(x_list,axis=1)
    train_X = np.concatenate([train_X,nor_X], axis=1)

train_y = train[target].values
print(train_X.shape, train_y.shape)

(32199, 10) (32199, 1)


In [24]:
test_X = minmax_scaler.transform(test[scale_features].values)

x_list = []
for col in nor_features:
    X_arr = np.array(test[col].apply(lambda x:np.array(x)).tolist())
    x_list.append(X_arr)
    
if len(nor_features) > 0:
    nor_X = np.concatenate(x_list,axis=1)
    test_X = np.concatenate([test_X,nor_X], axis=1)


test_y = test[target].values
print(test_X.shape, test_y.shape)

(600, 10) (600, 1)


In [25]:
#svc = LinearSVC(class_weight='balanced',random_state=10)
svc = RandomForestClassifier(n_estimators=200,n_jobs=4)
svc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=4, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [26]:
svc.fit(train_X,train_y)

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=4, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [27]:
pred = svc.predict(test_X)

In [28]:
pred.shape

(600,)

In [29]:
#test['pred'] = np.apply_along_axis(np.argmax,axis=1,arr=pred)
test['pred'] = np.apply_along_axis(output_change,axis=1,arr=pred.reshape(-1,1))
#test['pred'] = pred
true = np.apply_along_axis(output_change, axis=1,arr=test_y)

In [30]:
test['pred'].value_counts()

0    425
1    175
Name: pred, dtype: int64

In [31]:
print(log_loss(true,pred), accuracy_score(true,test['pred'].tolist()))

7.31087425305 0.988333333333


In [32]:
eval_metric = evaluation(df['swell(t+1)'].apply(output_change))

In [33]:
eval_metric.evaluate(test['pred'])

1    419
2    174
0      7
dtype: int64


767

In [34]:
#import xgboost

In [60]:
def sliding_pred(train_df, test_df):
    test_day = set(map(lambda x:str(int(x)//100),test_df.index.tolist()))
    
    pred_result = []
    
    for day in test_day:
        minmax_scaler = MinMaxScaler(copy=True, feature_range=(0,1))
        
        train_et = str(int(get_prev_date(day))+23)
        train = train_df.loc[:train_et]
        minmax_scaler.fit(train[scale_features].dropna())
        train_X = minmax_scaler.transform(train[scale_features].values)
        
        
        x_list = []
        for col in nor_features:
            X_arr = np.array(train[col].apply(lambda x:np.array(x)).tolist())
            x_list.append(X_arr)

            
        if len(nor_features) > 0:
            nor_X = np.concatenate(x_list,axis=1)
            train_X = np.concatenate([train_X,nor_X], axis=1)
        
        
        train_y = train[target].values
        
        print(train_et, train_X.shape, train_y.shape)
        
        #svc = xgboost.XGBClassifier()
        #svc = LinearSVC(class_weight='balanced',random_state=10)
        svc = RandomForestClassifier(n_estimators=200, n_jobs=4)
        svc.fit(train_X,train_y.reshape(-1,1))
        
        prev_swell = 0
        for hour in range(0,24):
            time = str(int(day)*100 + hour)
            test = test_df.loc[time]
            if not hour == 0:
                test['swell'] = prev_swell
                
            test_X = minmax_scaler.transform(test[scale_features].values).reshape(-1,len(scale_features))
            
            
            x_list = []
            for col in nor_features:
                X_arr = np.array(test[col])
                x_list.append(X_arr.reshape(-1,2))
            
            if len(x_list) > 0:
                nor_X = np.concatenate(x_list,axis=1)
                test_X = np.concatenate([test_X,nor_X], axis=1)
            
            test_y = test[target].values
            
            print(time, test_X.shape, test_y.shape)
            
            prev_swell = svc.predict(test_X.reshape(1,test_X.shape[1]))[0]
            pred_result.append(, prev_swell)
            
    return pred_result

In [36]:
%%time
preds = sliding_pred(train, test)

2016030223 (17140, 10) (17140, 1)


/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/ipykernel_launcher.py:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/skl

2016030300 (1, 10) (1,)
2016030301 (1, 10) (1,)


/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


2016030302 (1, 10) (1,)
2016030303 (1, 10) (1,)


...

2017060218 (1, 10) (1,)
2017060219 (1, 10) (1,)


/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


2017060220 (1, 10) (1,)
2017060221 (1, 10) (1,)


/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


2017060222 (1, 10) (1,)
2017060223 (1, 10) (1,)
CPU times: user 1min 52s, sys: 4.98 s, total: 1min 57s
Wall time: 1min 52s


In [51]:
np.sum(accs) / len(accs)

0.73999999999999999

In [52]:
len(preds)

600

In [53]:
test['ori_pred']  = preds
test['pred'] = np.apply_along_axis(output_change,axis=1,arr=np.array(preds).reshape(-1,1))

In [54]:
eval_metric = evaluation(df['swell(t+1)'].apply(output_change))

In [55]:
test['ori_pred'].value_counts()

0.0    324
1.0    174
2.0    102
Name: ori_pred, dtype: int64